
거리와 유사도
거리표현 방법
유클리드거리

레벤슈타인 거리:
자카드 유사도
크기가 0~1이 나옴

BoW 표현 -> 3차원으로 거리구하기
문장이 긴 것은 긴것들과 유사하다는 결론이 나서 유클리드거리를 접목하기 부적합

코사인 유사도
방향으로 유사도를 생각..

### 단어임베딩
희소표현 (원핫인코딩)
밀집표현 ; 벡터의 차원을 사용자가 설정한 단어의 차원으로 숫자를 표현, 숫자값을 가지게 됨

### 워드투벡터
-CBOW
주변단어로 중심단어 맞히기
-SkipGram
중심단어로 주변단어 맞히기

투자층

### fasttext
워투벡; 단어끼리 비교
얘는 준단어 토큰화
단어를 다 풀고 잘라서 토큰화 -> 성능 꽤 개선됨

단어의 의미는 별로 없다
신조어나 쪼갤 수 없는 단어 처리 가능


문장,글자를 벡터로 표현하는 것이 단어임베딩


# 단어임베딩 감성분석 실습

영화데이터 가져오기

In [2]:
import pandas as pd
import requests

res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')
with open('ratings_train.txt', 'wb') as f:
    f.write(res.content)
    
nsmc = pd.read_csv('ratings_train.txt', sep='\t')

In [3]:
nsmc

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 전처리

In [6]:
import re #정규표현식

# 정규화 함수 - 한글만 찾아주기
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [9]:
nsmc.loc[0, 'document']

'아 더빙.. 진짜 짜증나네요 목소리'

In [10]:
find_hangul(nsmc.loc[0, 'document'])

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [11]:
# 데이터 누락된 것들 제외하고 뽑기
nsmc[nsmc['document'].notnull()]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [14]:
# map함수 사용해서 함수 적용
data =nsmc[nsmc['document'].notnull()]['document'].map(find_hangul)

In [15]:
data[0]

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [16]:
def only_hangul(text):
    return ' '.join(find_hangul(text))

In [17]:
only_hangul(nsmc.loc[0, 'document'])

'아 더빙 진짜 짜증나네요 목소리'

In [18]:
data2 = nsmc[nsmc['document'].notnull()]['document'].map(only_hangul)

In [19]:
data2[0]

'아 더빙 진짜 짜증나네요 목소리'

In [20]:
with open('nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(data2))

(아까 했던) 단어별벡터로 만드는걸(단어를 모형에 설정해서 임베딩 계층만들어서 모델에 태움) fastText라이브러리로 만들어보자는 이야기

## FastText 모형 학습

In [21]:
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText

In [22]:
# sg=0: 기본값,cbow학습, sg=1: skip-gram학습
# alpha = 0.025: 학습률
# min_alpha : 학습률을 낮춰주는 역할 (조금 더 디테일하게 학습)
# window = 5 : 좌우 몇개 단어 맥락으로 학습을 할지 결정
# min_count = 5 : 최소 몇 번 나와야 학습을 함
# vector_size = 100 (기본값)

model = FastText(vector_size=16) # vector_size가 아까 설정했던 8과 동일한 기능

In [23]:
# vocab 설정
model.build_vocab(corpus_iterable=data)

In [26]:
# 학습
model.train(corpus_iterable=data,
            epochs=5, 
            total_examples=model.corpus_count,
            total_words = model.corpus_total_words)

(3998901, 5829395)

## 저장

In [27]:
model.save('nsmc.fasttext')

## 임베딩

In [28]:
# 저장한 모형 불러와서 임베딩
model = FastText.load('nsmc.fasttext')

In [29]:
model.wv.key_to_index #딕셔너리 형태

{'영화': 0,
 '너무': 1,
 '정말': 2,
 '진짜': 3,
 '이': 4,
 '그냥': 5,
 '왜': 6,
 '이런': 7,
 '더': 8,
 '점': 9,
 '수': 10,
 '영화를': 11,
 '다': 12,
 '잘': 13,
 '좀': 14,
 '보고': 15,
 'ㅋㅋ': 16,
 '그': 17,
 '영화가': 18,
 '영화는': 19,
 '본': 20,
 '봤는데': 21,
 '최고의': 22,
 '아': 23,
 '이건': 24,
 '내가': 25,
 '드라마': 26,
 '없는': 27,
 '없다': 28,
 '평점': 29,
 '완전': 30,
 '이렇게': 31,
 '참': 32,
 '이거': 33,
 '그리고': 34,
 '이게': 35,
 '좋은': 36,
 '있는': 37,
 '연기': 38,
 '내': 39,
 '평점이': 40,
 '보는': 41,
 '최고': 42,
 '다시': 43,
 '역시': 44,
 '스토리': 45,
 '쓰레기': 46,
 'ㅋ': 47,
 '난': 48,
 '많이': 49,
 '것': 50,
 '한': 51,
 'ㅋㅋㅋ': 52,
 '재밌게': 53,
 '없고': 54,
 '또': 55,
 '하는': 56,
 '아깝다': 57,
 '꼭': 58,
 '보면': 59,
 '마지막': 60,
 '가장': 61,
 '뭐': 62,
 '영화다': 63,
 '무슨': 64,
 '하지만': 65,
 '같은': 66,
 'ㅎㅎ': 67,
 '와': 68,
 '별로': 69,
 '작품': 70,
 '솔직히': 71,
 '끝까지': 72,
 '볼': 73,
 '넘': 74,
 '안': 75,
 '대한': 76,
 '만든': 77,
 '봐도': 78,
 '그래도': 79,
 '시간': 80,
 '같다': 81,
 '전혀': 82,
 '좋다': 83,
 '말이': 84,
 '지금': 85,
 '별': 86,
 '아주': 87,
 '근데': 88,
 '중': 89,
 '뭔가': 90,
 '영화의': 91,
 '하

In [30]:
'히어로' in model.wv.key_to_index #True는 임베딩 값이 있다는 뜻

True

In [31]:
# '히어로'의 임베딩 값 보기
model.wv['히어로']

array([-0.4200688 ,  0.5699441 ,  0.30497772,  0.37189355,  0.7739204 ,
        0.133562  , -0.7416958 , -0.40351447,  0.23216231,  0.13905022,
        0.11799232, -0.8794707 ,  0.3347639 , -0.3285863 , -0.07012366,
        0.53711104], dtype=float32)

In [32]:
len(model.wv['히어로'])

16

In [33]:
'슈퍼히어로' in model.wv.key_to_index

False

In [34]:
# 슈퍼히어로 단어 없는데 임베딩 값이 나온다 
# 이유: ngram단위로 잘라서 임베딩 값 만들기때문
# 슈퍼로 자르고 히어로로 자르고 이런식으로 잘라서 임베딩을 만들어낸다
# 글자가 없더라도 알아서 만든다

model.wv['슈퍼히어로']

array([-0.15870595,  0.25810537,  0.167094  ,  0.16181614,  0.29481593,
        0.07980631, -0.2835923 , -0.20157222,  0.14219771,  0.03472785,
        0.09746785, -0.32271406,  0.08526098, -0.13481358,  0.01896959,
        0.26480347], dtype=float32)

기존에는 숫자로 만들때 원핫벡터 - 자리 너무 많이 차지함
=>실수로 단어 표시하도록 하자 (->그럼 단어간의 관계성유사도도 알 수 있다) :단어 임베딩 


지정한 16벡터로 만들었음 
기존에 훈련되어있지 않은 단어에도 잘라서 임베딩 값을 만들어내더라..엔그램단위로 잘라서 그 값이 어딘가에 있으니 그 값을 가져옴
=> 없는 단어도 임베딩 값이 나옴
==> 값이 나왔으니 다른 단어랑 비교 가능

## 유사도
코사인유사도

In [35]:
# 코사인유사도로 비교
model.wv.similarity('슈퍼히어로', '히어로') 

0.98380053

In [36]:
model.wv.most_similar('평론가') # 평론가가 언급되는 문장에서 자주 나오는 단어들이 나와버림

[('점이냐', 0.9899165034294128),
 ('점이상은', 0.9897681474685669),
 ('점이상', 0.9875309467315674),
 ('점이하', 0.9875196814537048),
 ('점이라', 0.987502932548523),
 ('점이나', 0.9872541427612305),
 ('점대지', 0.9866974353790283),
 ('점이면', 0.986498236656189),
 ('점이야', 0.9860305190086365),
 ('점대면', 0.9859564304351807)]

## FastText를 이용한 감성분석

In [38]:
from gensim.models.fasttext import FastText

ft = FastText.load('nsmc.fasttext')

In [39]:
nsmc = pd.read_csv('ratings_train.txt', sep='\t')

In [41]:
# 데이터가 있는 것만 추출
df = nsmc[nsmc['document'].notnull()]
df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [42]:
from sklearn.model_selection import train_test_split

doc_train, doc_test, y_train, y_test = train_test_split(df['document'], df['label'], test_size=0.2, random_state=42)

In [43]:
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [46]:
# 아까처럼 평균내서 처리해야함(풀링1d) 신경망에선 자동이지만 여기선 수동으로
import numpy as np
x_train = np.zeros((1000,16))

# doc_train 너무 많아서 단어 1000개로 잘라서 16벡터로 만들겠다는 뜻

In [47]:
doc_train.shape

(119996,)

In [48]:
doc_train

31989            아 꿀잼ㅋ 친구랑 봤는데 너무 웃겼음 그리구 김우빈 잘생겼다..
63462                    개건의 졸작 스릴러? 스릴러라고 하게에도 민망하군
17518               장하나 한윤찬 제발 이어주세요 말도안되게 왜 설도현과ㅜㅜㅜ
123410                                   애로영화계의 개OOO
104181                               내용이나 그래픽자체가 허접함
                             ...                    
119882      꿈을 꾸는사람 꿈을 이룬사람 돌멩이도 꿈은 있잖아! 꿈과희망을 주는 영화
103696                                레니 할린.. 이게 뭐니?
131936                                    시라노; 연애조작단
146872                                    집중이 쉽지 않다.
121961    엔딩 장면이 좋고 소소한 일상과 고민과 연애가 공감을 주어 재밌게 보았어여~
Name: document, Length: 119996, dtype: object

In [54]:
for i, doc in enumerate(doc_train.iloc[:1000]):
#     for word in find_hangul(doc):
#         print(ft.wv[word]) #단어 하나하나의 임베딩벡터 프린트
    
    # list compresion으로
    vs = [ft.wv[word] for word in find_hangul(doc) if word in ft.wv]
    
    # 단어들을 돌리면서 열로 평균을 낸다
    if vs:
        x_train[i,] = np.mean(vs, axis=0) # 열로 평균. #1000개가 나옴
        
    else:
        pass

리스트하나 : 한 문장
vs : 하나하나 임베딩된 벡터
vs개수 : 6/ 문장 안의 단어가 6개
vs[0] : 0번째 단어의 임베딩 벡터들(16벡터)
16개짜리 벡터가 6개가 있다~~
-> 열로 평균 np.mean(vs, axis=0) (아까 했던 pooling1D같은)

==> 6*16 의 평균이 16개로 나왔다


In [56]:
x_train.shape

(1000, 16)

순서
1.FastText로 영화댓글 훈련: 단어별 16 벡터로 변환 (임베딩 벡터)
나온 값들로 유사도 보기 가능

2.FastText 감성분석 영화댓글
문서 각각의 댓글들 숫자로 인코딩
방법들
- 원핫인코딩 해서 모델에 밀어넣기
- 패딩해서 밀어넣기
- 단어간 유사성 표현하고 있는 임베딩 벡터 사용
    한 문서 안의 단어 하나당 각각 임베딩 16벡터 만들어서
    16벡터를 세로로(열로) 평균값내기 => 16개 벡터 나옴
    (단어개수*16벡터)/단어개수 => 16백터나옴
    
    -> 공간 만들어놓은 x_train에 넣기
        그걸로 문서를 파악한다.
        이게 fit하는 시점의 x_train이 된다 나중에 이걸 밀어넣는다. 
        
아까 2001개 8개는 알아서 셋팅을 해줬더라 신경망에서... 